In [ ]:
pip install keras-tuner

In [3]:
import mlflow
import mlflow.keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.src.legacy.preprocessing.image import ImageDataGenerator
from kerastuner import HyperModel, RandomSearch
import os
from kerastuner import HyperParameters
# Установите URI для отслеживания MLflow
mlflow.set_tracking_uri('http://127.0.0.1:5000')

C:\Users\User\AppData\Local\Temp\ipykernel_17808\2648883498.py:6: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperModel, RandomSearch


In [4]:
classes = os.listdir('augmented')

In [ ]:
# Определите вашу модель CNN
def create_model(filters_1=32, filters_2=64, filters_3=128, dropout_rate=0.5, learning_rate=0.001):
    model = Sequential()
    model.add(Conv2D(filters_1, (3, 3), activation='relu', input_shape=(100, 100, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(filters_2, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(filters_3, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(len(classes), activation='softmax'))  # Предполагается 10 классов
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [5]:
import matplotlib.pyplot as plt

In [71]:
# Функция для обучения модели и логирования в MLflow
def train_model(filters_1=32, filters_2=64, filters_3=128,
                dropout_rate=0.5, learning_rate=0.001,
                batch_size=32, epochs=10):
    
    with mlflow.start_run():
        # Создание модели с заданными гиперпараметрами
        model = create_model(filters_1=filters_1,
                             filters_2=filters_2,
                             filters_3=filters_3,
                             dropout_rate=dropout_rate,
                             learning_rate=learning_rate)
        
        # Подготовка данных из папки augmented
        datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2) 
        
        train_generator = datagen.flow_from_directory(
            'augmented',
            target_size=(100, 100),
            batch_size=batch_size,
            shuffle = True,
            class_mode='categorical',
            subset='training'
        )
        
        validation_generator = datagen.flow_from_directory(
            'augmented',
            target_size=(100, 100),
            batch_size=batch_size,
            shuffle = True,
            class_mode='categorical',
            subset='validation'
        )
        
        # Обучение модели с логированием в MLflow и метрик после каждой эпохи
        history = model.fit(train_generator,
                            validation_data=validation_generator,
                            epochs=epochs) 
        
        # Логирование модели в MLflow
        mlflow.keras.log_model(model, "model")

        # Логирование гиперпараметров после завершения обучения
        mlflow.log_param("batch_size", batch_size)
        mlflow.log_param("filters_1", filters_1)
        mlflow.log_param("filters_2", filters_2)
        mlflow.log_param("filters_3", filters_3)
        mlflow.log_param("dropout_rate", dropout_rate)
        mlflow.log_param("learning_rate", learning_rate)

        for epoch in range(len(history.history['val_accuracy'])):
            mlflow.log_metric("val_accuracy", history.history['val_accuracy'][epoch], step=epoch)
            mlflow.log_metric("accuracy", history.history['accuracy'][epoch], step=epoch)
            mlflow.log_metric("val_loss", history.history['val_loss'][epoch], step=epoch)  # Сохраняем val_loss
            mlflow.log_metric("loss", history.history['loss'][epoch], step=epoch)  # Сохраняем loss

        # Создание графика для accuracy
        plt.figure(figsize=(6, 4))
        plt.plot(history.history['accuracy'], label='accuracy')
        plt.plot(history.history['val_accuracy'], label='val_accuracy')
        plt.title('Model Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend(loc='upper left')

        # Сохранение графика accuracy как изображения
        accuracy_plot_file_path = "accuracy_plot.png"
        plt.savefig(accuracy_plot_file_path)
        
        # Закрытие графика
        plt.close()

        # Логирование изображения accuracy в MLflow
        mlflow.log_artifact(accuracy_plot_file_path)

        
        # Создание графика для loss
        plt.figure(figsize=(6, 4))
        plt.plot(history.history['loss'], label='loss')
        plt.plot(history.history['val_loss'], label='val_loss')
        plt.title('Model Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend(loc='upper left')

        # Сохранение графика loss как изображения
        loss_plot_file_path = "loss_plot.png"
        plt.savefig(loss_plot_file_path)
        
        # Закрытие графика
        plt.close()

         # Логирование изображения loss в MLflow
        mlflow.log_artifact(loss_plot_file_path)

In [ ]:
import itertools
# Списки параметров для перебора
filters_1_list = [16, 32]
filters_2_list = [32, 64]
filters_3_list = [64, 128]
dropout_rates = [0.5]
learning_rate = [0.001, 0.0001]
batch_sizes = [32, 64]
epochs_list = [5]

# Генерация всех возможных комбинаций параметров
param_combinations = itertools.product(filters_1_list,
                                        filters_2_list,
                                        filters_3_list,
                                        dropout_rates,
                                        learning_rate,
                                        batch_sizes,
                                        epochs_list)

# Обучение модели для каждой комбинации параметров
for params in param_combinations:
    filters_1, filters_2, filters_3, dropout_rate, learning_rate, batch_size, epochs = params
    
    print(f"Обучение модели с параметрами: "
          f"filters_1={filters_1}, "
          f"filters_2={filters_2}, "
          f"filters_3={filters_3}, "
          f"dropout_rate={dropout_rate}, "
          f"learning_rate={learning_rate},"
          f"batch_size={batch_size}, "
          f"epochs={epochs}")
    
    train_model(filters_1=filters_1,
                filters_2=filters_2,
                filters_3=filters_3,
                dropout_rate=dropout_rate,
                learning_rate=learning_rate,
                batch_size=batch_size,
                epochs=epochs)

print("Обучение завершено.")

# Без ImageGenerator

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
def create_model(dropout_rate=0.5, im_shape=(100, 100, 3), learning_rate=0.001):
    cnn_model = Sequential([
        # Первый сверточный слой
        Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=im_shape),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),  # Слой нормализации
        Dropout(dropout_rate),          # Слой дропаут

        # Второй сверточный слой
        # Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
        # MaxPooling2D(pool_size=(2, 2)),
        # BatchNormalization(),  # Слой нормализации
        # Dropout(dropout_rate),          # Слой дропаут

        Flatten(),             # Преобразование в одномерный вектор

        # Полносвязный слой
        Dense(1024, activation='relu'),

        # Выходной слой для классификации на 43 класса
        Dense(len(classes), activation='softmax')  
    ])
    
    optimizer = Adam(learning_rate=learning_rate)
    cnn_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return cnn_model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def create_model(dropout_rate=0.5, im_shape=(100, 100, 3), learning_rate=0.001):
    cnn_model = Sequential([
        Conv2D(filters=36, kernel_size=7, activation='relu', input_shape=im_shape),
        MaxPooling2D(pool_size=2),
        Conv2D(filters=54, kernel_size=5, activation='relu'),
        MaxPooling2D(pool_size=2),
        Flatten(),
        Dense(2024, activation='relu'),
        Dropout(dropout_rate),
        Dense(1024, activation='relu'),
        Dropout(dropout_rate),
        Dense(512, activation='relu'),
        Dropout(dropout_rate),
        Dense(len(classes), activation='softmax')  # Предполагается 20 классов
    ])
    optimizer = Adam(learning_rate=learning_rate)
    cnn_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return cnn_model

In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
# Определите вашу модель CNN
def create_model(filters_1=32, filters_2=64, filters_3=128, dropout_rate=0.5, learning_rate=0.001):
    model = Sequential()
    model.add(Conv2D(filters_1, (3, 3), activation='relu', input_shape=(100, 100, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(filters_2, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters_3, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(len(classes), activation='softmax'))  # Предполагается 10 классов
    
    optimazer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimazer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [7]:
from PIL import Image
import numpy as np

def load_image(file_path, target_size=(100, 100)):
    """
    Загружает изображение из указанного пути, изменяет его размер и нормализует пиксели.

    :param file_path: Путь к файлу изображения.
    :param target_size: Размер, до которого нужно изменить изображение (ширина, высота).
    :return: Нормализованное изображение в виде массива NumPy.
    """
    # Открываем изображение
    image = Image.open(file_path)

    # Изменяем размер изображения
    image = image.resize(target_size)

    # Преобразуем изображение в массив NumPy
    image_array = np.array(image)

    # Нормализация пикселей (если необходимо)
    # Приводим значения пикселей к диапазону [0, 1]
    image_array = image_array / 255.0

    return image_array

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

def train_model(X_train, X_test, y_train, y_test,
                dropout_rate=0.5, learning_rate=0.001,
                batch_size=32, epochs=10):
    
    with mlflow.start_run():
        # Создание модели с заданными гиперпараметрами
        model = create_model(dropout_rate=dropout_rate,
                             learning_rate=learning_rate)
        early_stopping = EarlyStopping(monitor='val_loss',  # Можно использовать 'val_accuracy' для мониторинга точности
                               patience=2,         # Количество эпох без улучшения перед остановкой
                               restore_best_weights=True)  # Восстановить лучшие веса
        reduce_lr = ReduceLROnPlateau(monitor='val_loss',  # Метрика для мониторинга
                               factor=0.5,          # На сколько уменьшать learning rate
                               patience=2,         # Количество эпох без улучшения перед уменьшением
                               min_lr=1e-6)        # Минимально допустимый learning rate
        
        # Обучение модели с логированием в MLflow и метрик после каждой эпохи
        history = model.fit(X_train,
                            y_train,
                            validation_data=(X_test, y_test),
                            epochs=epochs,
                            batch_size=batch_size,
                            callbacks=[early_stopping])
        
        # Логирование модели в MLflow
        mlflow.keras.log_model(model, "model")

        # Логирование гиперпараметров после завершения обучения
        mlflow.log_param("batch_size", batch_size)
        mlflow.log_param("dropout_rate", dropout_rate)
        mlflow.log_param("learning_rate", learning_rate)

        for epoch in range(len(history.history['val_accuracy'])):
            mlflow.log_metric("val_accuracy", history.history['val_accuracy'][epoch], step=epoch)
            mlflow.log_metric("accuracy", history.history['accuracy'][epoch], step=epoch)
            mlflow.log_metric("val_loss", history.history['val_loss'][epoch], step=epoch)  
            mlflow.log_metric("loss", history.history['loss'][epoch], step=epoch)  

        # Создание графика для accuracy
        plt.figure(figsize=(6, 4))
        plt.plot(history.history['accuracy'], label='accuracy')
        plt.plot(history.history['val_accuracy'], label='val_accuracy')
        plt.title('Model Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend(loc='upper left')

        # Сохранение графика accuracy как изображения
        accuracy_plot_file_path = "accuracy_plot.png"
        plt.savefig(accuracy_plot_file_path)
        
        # Закрытие графика
        plt.close()

        # Логирование изображения accuracy в MLflow
        mlflow.log_artifact(accuracy_plot_file_path)

    
        # Создание графика для loss
        plt.figure(figsize=(6, 4))
        plt.plot(history.history['loss'], label='loss')
        plt.plot(history.history['val_loss'], label='val_loss')
        plt.title('Model Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend(loc='upper left')

        # Сохранение графика loss как изображения
        loss_plot_file_path = "loss_plot.png"
        plt.savefig(loss_plot_file_path)

        # Закрытие графика 
        plt.close()

        # Логирование изображения loss в MLflow 
        mlflow.log_artifact(loss_plot_file_path)

In [30]:
del X
del y

In [31]:
X = []
y = []
data_dir = 'augmented'
for person_folder in os.listdir(data_dir):
    person_path = os.path.join(data_dir, person_folder)
    if os.path.isdir(person_path):  # Проверяем, что это папка
        for file_name in os.listdir(person_path):
            file_path = os.path.join(person_path, file_name)
            if file_name.endswith('.jpg') or file_name.endswith('.png'):  # Замените на нужные форматы
                # Здесь вы можете загрузить изображение или данные
                image = load_image(file_path)  # Замените на вашу функцию загрузки изображения
                X.append(image)
                y.append(person_folder)

X = np.array(X)
y = np.array(y)

In [32]:
len(X)

12400

In [33]:
del y_one_hot, X_train, X_test, y_train, y_test

In [34]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)  # sparse=False для получения массива NumPy вместо разреженной матрицы
y_one_hot = encoder.fit_transform(y.reshape(-1, 1))  # Преобразуем y в двумерный массив

print("Размер X:", X.shape)
print("Размер y (one-hot):", y_one_hot.shape)

Размер X: (12400, 100, 100, 3)
Размер y (one-hot): (12400, 31)


In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

print("Размер обучающей выборки X:", X_train.shape)
print("Размер тестовой выборки X:", X_test.shape)
print("Размер обучающей выборки y:", y_train.shape)
print("Размер тестовой выборки y:", y_test.shape)

Размер обучающей выборки X: (9920, 100, 100, 3)
Размер тестовой выборки X: (2480, 100, 100, 3)
Размер обучающей выборки y: (9920, 31)
Размер тестовой выборки y: (2480, 31)


In [43]:
import itertools

dropout_rates = [0.4, 0.3]
learning_rate = [0.0001]
batch_sizes = [32, 64]
epochs_list = [10]

# Генерация всех возможных комбинаций параметров
param_combinations = itertools.product(dropout_rates,
                                        learning_rate,
                                        batch_sizes,
                                        epochs_list)

# Обучение модели для каждой комбинации параметров
for params in param_combinations:
    dropout_rate, learning_rate, batch_size, epochs = params
    
    print(f"Обучение модели с параметрами: "
          f"dropout_rate={dropout_rate}, "
          f"learning_rate={learning_rate},"
          f"batch_size={batch_size}, "
          f"epochs={epochs}")
    
    train_model(X_train, X_test, y_train, y_test,
                dropout_rate=dropout_rate,
                learning_rate=learning_rate,
                batch_size=batch_size,
                epochs=epochs)

print("Обучение завершено.")

Обучение модели с параметрами: dropout_rate=0.4, learning_rate=0.0001,batch_size=32, epochs=10
Epoch 1/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 52s 157ms/step - accuracy: 0.2437 - loss: 3.0382 - val_accuracy: 0.1565 - val_loss: 2.9761
Epoch 2/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 47s 153ms/step - accuracy: 0.6723 - loss: 1.1088 - val_accuracy: 0.6944 - val_loss: 1.0675
Epoch 3/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 48s 153ms/step - accuracy: 0.8547 - loss: 0.4979 - val_accuracy: 0.8153 - val_loss: 0.6763
Epoch 4/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 48s 153ms/step - accuracy: 0.9374 - loss: 0.2390 - val_accuracy: 0.8452 - val_loss: 0.5475
Epoch 5/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 48s 154ms/step - accuracy: 0.9715 - loss: 0.1285 - val_accuracy: 0.8637 - val_loss: 0.5051
Epoch 6/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 48s 155ms/step - accuracy: 0.9799 - loss: 0.0908 - val_accuracy: 0.8730 - val_loss: 0.4667
Epoch 7/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 48s 155ms/step - accuracy: 0.9898 - loss: 0.0590 - val_accuracy: 0.8685 - val_l

2025/04/01 21:47:33 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/04/01 21:47:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run fearless-snail-50 at: http://127.0.0.1:5000/#/experiments/0/runs/b2938ffa77194b1492095df54188f399
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0
Обучение модели с параметрами: dropout_rate=0.4, learning_rate=0.0001,batch_size=64, epochs=10
Epoch 1/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 45s 271ms/step - accuracy: 0.2254 - loss: 3.1316 - val_accuracy: 0.0597 - val_loss: 3.5818
Epoch 2/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 42s 269ms/step - accuracy: 0.6243 - loss: 1.3057 - val_accuracy: 0.1089 - val_loss: 3.5060
Epoch 3/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 42s 273ms/step - accuracy: 0.7864 - loss: 0.7297 - val_accuracy: 0.5040 - val_loss: 1.7085
Epoch 4/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 42s 270ms/step - accuracy: 0.8886 - loss: 0.4069 - val_accuracy: 0.7476 - val_loss: 0.8964
Epoch 5/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 42s 269ms/step - accuracy: 0.9462 - loss: 0.2289 - val_accuracy: 0.8133 - val_loss: 0.6610
Epoch 6/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 42s 270ms/step - accuracy: 0.9672 - loss:

2025/04/01 21:55:34 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/04/01 21:55:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run rare-boar-299 at: http://127.0.0.1:5000/#/experiments/0/runs/489b382abf054da89ee7f019baeae21c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0
Обучение модели с параметрами: dropout_rate=0.3, learning_rate=0.0001,batch_size=32, epochs=10
Epoch 1/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 50s 153ms/step - accuracy: 0.2818 - loss: 2.7842 - val_accuracy: 0.1512 - val_loss: 3.2244
Epoch 2/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 51s 166ms/step - accuracy: 0.7527 - loss: 0.8598 - val_accuracy: 0.6810 - val_loss: 1.0734
Epoch 3/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 51s 163ms/step - accuracy: 0.9058 - loss: 0.3472 - val_accuracy: 0.8262 - val_loss: 0.6248
Epoch 4/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 51s 163ms/step - accuracy: 0.9673 - loss: 0.1551 - val_accuracy: 0.8460 - val_loss: 0.5337
Epoch 5/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 48s 154ms/step - accuracy: 0.9867 - loss: 0.0749 - val_accuracy: 0.8565 - val_loss: 0.4974
Epoch 6/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 48s 154ms/step - accuracy: 0.9893 - loss: 0.0

KeyboardInterrupt: 